In [456]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.options.mode.chained_assignment = None 

In [457]:
def strf_date(x):
    if pd.isnull(x): return ""
    str_x = str(int(float(x)))
    return str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:]

In [506]:
date_rows = ['Z71_OPEN_DATE',
             'Z30_PROCESS_STATUS_DATE',
             'Z30_UPDATE_DATE',
             'UPDATE_DATE',
             'Z68_OPEN_DATE',
             'Z68_ORDER_STATUS_DATE_X']

In [458]:
def clean_data(df):

    df = df.applymap(lambda x: x.strip() if type(x) == str else x)
    for d in date_rows:
        df[d] = df[d].apply(strf_date)

    df = df.reset_index(drop=True)
    df = df.drop_duplicates()
    barcode_duplicates = df['Z30_BARCODE'].duplicated(keep='last')
    barcode_duplicates.name = 'barcode_duplicates'
    df = df.join(barcode_duplicates)
    df = df[df['barcode_duplicates'] == False]
    df = df.drop(['barcode_duplicates'], axis=1)
    df['Z68_TOTAL_PRICE'] = df['Z68_TOTAL_PRICE'].fillna("")
    df['Z68_TOTAL_PRICE'] = df['Z68_TOTAL_PRICE'].apply(lambda x: "".join(x.split(',')))
    df = df.reset_index(drop=True)
    return df

In [459]:
prev = pd.read_excel("3_25.xlsx", dtype=str)
curr = pd.read_excel("4_1.xlsx", dtype=str)

In [460]:
prev.shape

(3538, 29)

In [461]:
curr.shape

(3586, 29)

In [462]:
prev = clean_data(prev)
curr = clean_data(curr)

In [463]:
prev.shape

(3530, 29)

In [464]:
curr.shape

(3578, 29)

In [465]:
not_nyush = prev[prev["Z68_ORDER_NUMBER"].str.contains("NYUSH") == 0]

In [466]:
prev = prev[prev["Z68_ORDER_NUMBER"].str.contains("NYUSH")]

In [467]:
curr = curr[curr["Z68_ORDER_NUMBER"].str.contains("NYUSH")]

In [468]:
prev["Z68_ORDER_NUMBER"] = prev["Z68_ORDER_NUMBER"].apply(lambda x: x[5:])

In [469]:
curr["Z68_ORDER_NUMBER"] = curr["Z68_ORDER_NUMBER"].apply(lambda x: x[5:])

In [470]:
from datetime import date
current_year = int(date.today().isoformat()[0:4])
year_dict = {i:None for i in range(current_year-3, current_year+1)}
for year in year_dict.keys():
    this_year = prev["Z68_ORDER_NUMBER"].apply(lambda x: x[0:4] == str(year))
    this_year.name = "this_year"
    current_df = prev.join(this_year)
    current_df = current_df[current_df["this_year"] == True]
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].apply(lambda x: x[4:])
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].astype(int)
    current_df = current_df.sort_values(by=["Z68_ORDER_NUMBER"])
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].astype(str)
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].apply(lambda x: "NYUSH" + str(year) + x)
    del current_df["this_year"]
    year_dict[year] = current_df

In [471]:
sorted_prev = pd.concat([year_df for year_df in year_dict.values()])
sorted_prev.reset_index(inplace=True, drop=True)
sorted_prev

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
0,006614620,Coloratura /,Arrival,2019-03-30,JONESL,1 new items reported.,31124065529548,,,01,BOOK,MAIN,2019-04-09,2019-04-09,LINL,2019-04-24,2019-03-25,NSHNG,CLS,C,F,NYUSH,NYUSH2019676,M,24.95,C,2019-04-25,YBPSH,Needed by April 18th. Requested by dp102@nyu.e...
1,001518361,The political lives of dead bodies :,Arrival,2019-03-29,JONESL,1 new items reported.,31124065529514,,,01,BOOK,MAIN,2019-04-09,2019-04-09,LINL,2019-04-04,2019-03-25,NSHNG,CLS,C,F,NYUSH,NYUSH2019677,M,28.80,C,2019-04-04,AMAZONSH,Requested by MLD9@nyu.edu via ILL TN795118
2,006614640,Pattern recognition /,Arrival,2019-04-02,LEEREB,1 new items reported.,31124065529746,,,01,BOOK,MAIN,2019-04-10,2021-02-04,LINL,2019-04-04,2019-03-26,NSHNG,CLS,C,F,NYUSH,NYUSH2019678,M,56.99,C,2019-04-04,AMAZONSH,"Has e, please order print. For Abrahao Bruno's..."
3,005685267,Hands-on machine learning with Scikit-Learn an...,Arrival,2019-03-29,JONESL,1 new items reported.,31124065529472,,,01,BOOK,MAIN,2019-04-10,2019-06-03,GENERICSH,2019-04-04,2019-03-26,NSHNG,CLS,C,F,NYUSH,NYUSH2019679,M,56.99,C,2019-04-04,AMAZONSH,For Abrahao Bruno's Spring 2019 course: Artifi...
4,006614641,Linear algebra and learning from data /,Arrival,2019-03-29,JONESL,1 new items reported.,31124065529480,,,01,BOOK,MAIN,2019-04-09,2021-02-04,LINL,2019-04-04,2019-03-26,NSHNG,CLS,C,F,NYUSH,NYUSH2019680,M,75.68,C,2019-04-04,AMAZONSH,For Abrahao Bruno's Spring 2019 course: Artifi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3524,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM
3525,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM
3526,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM
3527,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS


In [472]:
not_nyush

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
1166,008171380,Calligraphy copybook of autobiography /,Arrival,2021-09-06,HEW,1 new items reported.,31124068869875,,,01,BOOK,MAIN,2021-09-20,2021-09-20,XIAL,2021-09-07,2021-07-30,NSHNG,CLS,C,F,NYUSH,ON1227270024,M,128.00,C,2021-09-07,SBTSH,[CMKM]


In [429]:
exported_prev = pd.concat([not_nyush, sorted_prev])
exported_prev

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,barcode_duplicates
1166,008171380,Calligraphy copybook of autobiography /,Arrival,2021-09-06,HEW,1 new items reported.,31124068869875,,,01,BOOK,MAIN,2021-09-20,2021-09-20,XIAL,2021-09-07,2021-07-30,NSHNG,CLS,C,F,NYUSH,ON1227270024,M,128.00,C,2021-09-07,SBTSH,False
0,006614620,Coloratura /,Arrival,2019-03-30,JONESL,1 new items reported.,31124065529548,,,01,BOOK,MAIN,2019-04-09,2019-04-09,LINL,2019-04-24,2019-03-25,NSHNG,CLS,C,F,NYUSH,NYUSH2019676,M,24.95,C,2019-04-25,YBPSH,False
1,001518361,The political lives of dead bodies :,Arrival,2019-03-29,JONESL,1 new items reported.,31124065529514,,,01,BOOK,MAIN,2019-04-09,2019-04-09,LINL,2019-04-04,2019-03-25,NSHNG,CLS,C,F,NYUSH,NYUSH2019677,M,28.80,C,2019-04-04,AMAZONSH,False
2,006614640,Pattern recognition /,Arrival,2019-04-02,LEEREB,1 new items reported.,31124065529746,,,01,BOOK,MAIN,2019-04-10,2021-02-04,LINL,2019-04-04,2019-03-26,NSHNG,CLS,C,F,NYUSH,NYUSH2019678,M,56.99,C,2019-04-04,AMAZONSH,False
3,005685267,Hands-on machine learning with Scikit-Learn an...,Arrival,2019-03-29,JONESL,1 new items reported.,31124065529472,,,01,BOOK,MAIN,2019-04-10,2019-06-03,GENERICSH,2019-04-04,2019-03-26,NSHNG,CLS,C,F,NYUSH,NYUSH2019679,M,56.99,C,2019-04-04,AMAZONSH,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3524,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,False
3525,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,False
3526,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,False
3527,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,False


In [369]:
exported_prev.reset_index(drop=True, inplace=True)

In [371]:
exported_prev.to_excel("OUT_3_25.xlsx", index=False)

In [473]:
from datetime import date
current_year = int(date.today().isoformat()[0:4])
year_dict = {i:None for i in range(current_year-3, current_year+1)}
for year in year_dict.keys():
    this_year = curr["Z68_ORDER_NUMBER"].apply(lambda x: x[0:4] == str(year))
    this_year.name = "this_year"
    current_df = curr.join(this_year)
    current_df = current_df[current_df["this_year"] == True]
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].apply(lambda x: x[4:])
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].astype(int)
    current_df = current_df.sort_values(by=["Z68_ORDER_NUMBER"])
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].astype(str)
    current_df["Z68_ORDER_NUMBER"] = current_df["Z68_ORDER_NUMBER"].apply(lambda x: "NYUSH" + str(year) + x)
    del current_df["this_year"]
    year_dict[year] = current_df

In [474]:
sorted_curr = pd.concat([year_df for year_df in year_dict.values()])
sorted_curr.reset_index(inplace=True, drop=True)
sorted_curr

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3572,002806056,"Sonia Delaunay, art into fashion /",NaN,,NaN,NaN,2806056-10,OR,On Order,01,BOOK,MAIN,2022-03-31,2022-04-01,BEIB,2022-03-31,2022-04-01,NSHNG,SV,P,F,NYUSH,NYUSH20221381,M,32.78,P,2022-04-01,AMAZONSH,CMKM
3573,008030352,The book of sins /,NaN,,NaN,NaN,8030352-20,OR,On Order,01,BOOK,MAIN,2022-03-31,2022-04-01,BEIB,2022-03-31,2022-04-01,NSHNG,SV,P,F,NYUSH,NYUSH20221382,M,26.06,P,2022-04-01,AMAZONSH,CMKM
3574,003703395,20th century pattern design :,NaN,,NaN,NaN,3703395-10,OR,On Order,01,BOOK,MAIN,2022-03-31,2022-04-01,BEIB,2022-03-31,2022-04-01,NSHNG,SV,P,F,NYUSH,NYUSH20221383,M,99.95,P,2022-04-01,AMAZONSH,CMKM
3575,003563845,Ai Weiwei :,NaN,,NaN,NaN,3563845-10,OR,On Order,01,BOOK,MAIN,2022-03-31,2022-04-01,BEIB,2022-03-31,2022-04-01,NSHNG,SV,P,F,NYUSH,NYUSH20221384,M,44.04,P,2022-04-01,AMAZONSH,CMKM sensitive


In [475]:
prev_start = sorted_prev[sorted_prev["Z68_ORDER_NUMBER"] == sorted_curr.iloc[0]["Z68_ORDER_NUMBER"]]
start_idx = prev_start.iloc[0].name
start_idx

9

In [476]:
curr_end = sorted_curr[sorted_curr["Z68_ORDER_NUMBER"] == sorted_prev.iloc[-1]["Z68_ORDER_NUMBER"]]
end_idx = curr_end.iloc[-1].name
end_idx

3517

In [477]:
check_prev = sorted_prev.iloc[int(start_idx):]
check_curr = sorted_curr.iloc[:int(end_idx) + 1]

In [478]:
check_prev.reset_index(inplace=True, drop=True)
check_curr.reset_index(inplace=True, drop=True)

In [479]:
print(check_curr.shape, check_prev.shape)

(3518, 29) (3520, 29)


In [480]:
check_curr

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM
3514,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM
3515,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM
3516,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS


In [481]:
check_prev.reset_index(inplace=True)
check_curr.insert(check_curr.shape[1], "id", "")
check_curr.insert(check_curr.shape[1], "checked", False)

In [482]:
check_curr

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE,id,checked
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],,False
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],,False
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],,False
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589,,False
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM,,False
3514,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM,,False
3515,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM,,False
3516,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS,,False


In [483]:
to_del = check_prev.iloc[:0,:].copy()
to_del

,index,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE


In [484]:
deleted_rows = 0
for idx, row in check_prev.iterrows():
    if (check_curr.iloc[idx - deleted_rows]["BSN"] == row["BSN"] and 
        check_curr.iloc[idx - deleted_rows]["Z68_ORDER_NUMBER"] == row["Z68_ORDER_NUMBER"]):
            check_curr.at[idx - deleted_rows, "id"] = row["index"]
            check_curr.at[idx - deleted_rows, "checked"] = True
    else:
        filtered_curr = check_curr[(check_curr["BSN"] == row["BSN"])
                                    & (check_curr["Z68_ORDER_NUMBER"] == row["Z68_ORDER_NUMBER"])
                                    & (check_curr["checked"] == False)]
        if filtered_curr.shape[0] == 0:
            to_del = to_del.append(row)
            deleted_rows += 1
        elif filtered_curr.shape[0] == 1:
            idx_in_curr = filtered_curr.iloc[0].name
            check_curr.at[idx_in_curr, "id"] = row["index"]
            check_curr.at[idx_in_curr, "checked"] = True
        elif filtered_curr.shape[0] > 1:
            filtered_curr["distance"] = filtered_curr.index.map(lambda x: abs(x - int(row.name)))
            filtered_curr = filtered_curr.sort_values(by=["distance"])
            min_distance_idx = filtered_curr.iloc[0].name
            check_curr.at[min_distance_idx, "id"] = row["index"]
            check_curr.at[min_distance_idx, "checked"] = True

In [485]:
print(count_direct, count_matching)

3517 9


In [486]:
to_del

,index,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
2381,2381,003513319,The parasite /,NaN,,NaN,NaN,3513319-10,OR,On Order,01,BOOK,MAIN,2021-07-30,2021-07-30,HEW,2021-07-30,2021-07-30,NSHNG,SV,P,F,NYUSH,NYUSH20212381,M,109.00,P,2021-07-30,CEPIECSH,CMKM
2626,2626,001740427,"Japanese pride, American prejudice :",NaN,,NaN,NaN,1740427-10,OR,On Order,01,BOOK,MAIN,2021-09-02,2021-09-02,HEW,2021-09-02,2021-09-02,NSHNG,SV,P,F,NYUSH,NYUSH20212971,M,601.00,P,2021-09-02,BWSH,CMKM


In [487]:
check_curr["id"] = check_curr["id"].astype(str)

In [488]:
check_curr

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE,id,checked
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],0,True
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],1,True
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],2,True
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589,3,True
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM,3515,True
3514,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM,3516,True
3515,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM,3517,True
3516,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS,3518,True


In [489]:
check_prev

,index,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE
0,0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
1,1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
2,2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX]
3,3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589
4,4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,3515,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM
3516,3516,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM
3517,3517,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM
3518,3518,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS


In [490]:
to_insert = check_curr[check_curr["checked"] == False]
check_curr = check_curr[check_curr["checked"] == True]

In [491]:
del check_curr["checked"]
del to_insert["id"]
del to_insert["checked"]
to_insert = pd.concat([to_insert, sorted_curr.iloc[end_idx + 1:]])

In [496]:
to_insert.shape

(59, 29)

In [494]:
check_curr.shape

(3518, 30)

In [502]:
check_curr

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE,id
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],0
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],1
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],2
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589,3
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,006604134,"Karl Blossfeldt, 1865-1932 :",NaN,,NaN,NaN,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM,3515
3514,008617020,All roads lead north :,NaN,,NaN,NaN,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM,3516
3515,002520444,Up to the mountains and down to the villages :,NaN,,NaN,NaN,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM,3517
3516,008617026,Great challenges and even greater opportunities :,NaN,,NaN,NaN,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS,3518


In [504]:
def prepare_for_db(df):
    df = df.fillna(np.nan).replace([np.nan], [None])
    for row in date_rows:
        df[row] = df[row].replace([""], [None])
    df['Z68_TOTAL_PRICE'] = df['Z68_TOTAL_PRICE'].replace(["", None])

,BSN,Z13_TITLE,Z71_TEXT,Z71_OPEN_DATE,Z71_USER_NAME,Z71_DATA,Z30_BARCODE,Z30_ITEM_PROCESS_STATUS,ITEM_PROCESS_STATUS,Z30_ITEM_STATUS,Z30_MATERIAL,Z30_COLLECTION,Z30_PROCESS_STATUS_DATE,Z30_UPDATE_DATE,Z30_CATALOGER,UPDATE_DATE,Z68_OPEN_DATE,Z68_SUB_LIBRARY,Z68_ORDER_STATUS,Z68_INVOICE_STATUS,Z68_MATERIAL_TYPE,Z68_ORDERING_UNIT,Z68_ORDER_NUMBER,Z68_ORDER_TYPE,Z68_TOTAL_PRICE,Z68_ARRIVAL_STATUS,Z68_ORDER_STATUS_DATE_X,Z68_VENDOR_CODE,Z68_LIBRARY_NOTE,id
0,006621612,Wangzi he qiong haizi =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529985,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019732,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],0
1,006621614,An Mo =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529977,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019733,M,12.75,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],1
2,006621615,Di xin you ji =,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529951,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019734,M,12.95,C,2019-05-14,AMAZONSH,Requested by jas58 [QX],2
3,006621617,Hybrid humour :,Arrival,2019-04-13,JONESL,1 new items reported.,31124065530561,,,01,BOOK,MAIN,2019-06-06,2019-06-06,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019735,M,64.44,C,2019-05-14,AMAZONSH,Requested by mc7214@nyu.edu via ILL TN 796589,3
4,004488820,Environmental economics /,Arrival,2019-04-05,JONESL,1 new items reported.,31124065529993,,,01,BOOK,MAIN,2019-04-23,2019-04-23,GENERICSH,2019-05-13,2019-04-02,NSHNG,CLS,C,F,NYUSH,NYUSH2019736,M,126.18,C,2019-05-14,AMAZONSH,Requested by sz67@nyu.edu via ILL TN 797009,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513,006604134,"Karl Blossfeldt, 1865-1932 :",None,,None,None,6604134-20,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221266,M,145.20,P,2022-03-18,AMAZONSH,CMKM,3515
3514,008617020,All roads lead north :,None,,None,None,8617020-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221267,M,37.53,P,2022-03-18,AMAZONSH,CMKM,3516
3515,002520444,Up to the mountains and down to the villages :,None,,None,None,2520444-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221268,M,27.95,P,2022-03-18,ALIBRISSH,CMKM,3517
3516,008617026,Great challenges and even greater opportunities :,None,,None,None,8617026-10,OR,On Order,01,BOOK,MAIN,2022-03-18,2022-03-18,BEIB,2022-03-18,2022-03-18,NSHNG,SV,P,F,NYUSH,NYUSH20221269,M,128.86,P,2022-03-18,AMAZONSH,JAS,3518
